In [1]:
import numpy as np
import pandas as pd

In [2]:
produk = pd.read_csv('Data Produk Properti.csv')
produk.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Nama Asuransi   1000 non-null   object 
 1   kategori        1000 non-null   object 
 2   produk_id       1000 non-null   object 
 3   user_id         1000 non-null   object 
 4   Rating          1000 non-null   float64
 5   Premi Asuransi  1000 non-null   float64
dtypes: float64(2), object(4)
memory usage: 47.0+ KB


In [3]:
produk

,Nama Asuransi,kategori,produk_id,user_id,Rating,Premi Asuransi
0,BCA Asuransi Rumah Tinggal,Properti,P0001,U0053,4.0,7.740000e+05
1,BCA Asuransi Properti Komersial,Properti,P0002,U0142,4.3,7.740000e+05
2,BCA Asuransi Kebakaran,Properti,P0003,U0112,4.5,7.690000e+05
3,Asuransi Simas Ruko,Properti,P0004,U0153,3.4,2.000000e+05
4,Simas Rumah Hemat Plus,Properti,P0005,U0147,4.4,2.000000e+05
...,...,...,...,...,...,...
995,Simas Asuransi Kebakaran Property All Risks/In...,Properti,P0996,U0088,3.5,1.785000e+06
996,AXA Mandiri Asuransi Rumahku Plus,Properti,P0997,U0166,3.5,6.574286e+05
997,ACA Asuransi Rumah Idaman (ASRI),Properti,P0998,U0171,3.4,6.574286e+05
998,ACA Asuransi Kebakaran (Standar OJK),Properti,P0999,U0054,4.3,6.574286e+05


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from sklearn.preprocessing import StandardScaler

In [5]:
# Mapping user dan produk ke index
user_ids = produk['user_id'].unique()
produk_ids = produk['produk_id'].unique()

user2idx = {u: i for i, u in enumerate(user_ids)}
produk2idx = {p: i for i, p in enumerate(produk_ids)}

produk['user_idx'] = produk['user_id'].map(user2idx)
produk['produk_idx'] = produk['produk_id'].map(produk2idx)

# Fitur input
user_indices = produk['user_idx'].values
produk_indices = produk['produk_idx'].values
premi = produk[['Premi Asuransi']].values
ratings = produk['Rating'].values

# Normalisasi premi
from sklearn.preprocessing import StandardScaler
scaler_premi = StandardScaler()
premi_scaled = scaler_premi.fit_transform(premi)

# Info jumlah
num_users = len(user2idx)
num_products = len(produk2idx)
embedding_dim = 32


In [6]:
# Inputs
user_input = Input(shape=(), dtype=tf.int32, name='user_input')
produk_input = Input(shape=(), dtype=tf.int32, name='produk_input')
premi_input = Input(shape=(1,), dtype=tf.float32, name='premi_input')

# Embedding
user_emb = layers.Embedding(num_users, embedding_dim)(user_input)
produk_emb = layers.Embedding(num_products, embedding_dim)(produk_input)

# Flatten
user_vec = layers.Flatten()(user_emb)
produk_vec = layers.Flatten()(produk_emb)

# Concatenate semua fitur
x = layers.Concatenate()([user_vec, produk_vec, premi_input])

# Dense layers
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
output = layers.Dense(1)(x)

# Model
model = Model(inputs=[user_input, produk_input, premi_input], outputs=output)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ produk_input        │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 32)        │      6,400 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 32)        │     32,000 │ produk_input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 32)        │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 32)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ premi_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 65)        │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0],  │
│                     │                   │            │ premi_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,224 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         33 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 44,737 (174.75 KB)

 Trainable params: 44,737 (174.75 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.fit(
    [user_indices, produk_indices, premi_scaled],
    ratings,
    epochs=10,
    batch_size=64,
    validation_split=0.1
)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 15.9760 - val_loss: 14.2836
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 13.6156 - val_loss: 11.6628
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 10.3172 - val_loss: 7.9502
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 6.0801 - val_loss: 3.6353
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.0160 - val_loss: 1.4111
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.8416 - val_loss: 1.1396
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.4456 - val_loss: 1.1699
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2664 - val_loss: 1.0338
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1717 - val_loss: 1.0629
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0905 - val_loss: 1.0899


In [8]:
# Ambil produk unik berdasarkan produk_idx
produk_unique = produk.drop_duplicates(subset='produk_idx').set_index('produk_idx')

In [9]:
def recommend_hybrid_by_user_premi(user_id, premi_user, top_n=10):
    # Scaling premi input user
    premi_user_scaled = scaler_premi.transform([[premi_user]])[0][0]

    # Jika user lama
    if user_id in user2idx:
        user_idx = user2idx[user_id]
        rated_produk_idx = produk[produk['user_id'] == user_id]['produk_idx'].unique()
        unrated_produk_idx = np.setdiff1d(np.arange(len(produk2idx)), rated_produk_idx)
    else:
        print(f"User {user_id} tidak ditemukan, gunakan cold-start berbasis premi user.")
        user_idx = 0  # dummy
        unrated_produk_idx = np.arange(len(produk2idx))
    
    user_array = np.array([user_idx] * len(unrated_produk_idx))
    produk_array = np.array(unrated_produk_idx)
    premi_array = np.array([premi_user_scaled] * len(unrated_produk_idx))

    # Prediksi rating
    pred_ratings = model.predict([user_array, produk_array, premi_array], verbose=0).flatten()

    # Gabungkan hasil prediksi
    rekom_df = pd.DataFrame({
        'produk_idx': produk_array,
        'Rating': pred_ratings  # <- rename ke Rating untuk tampilan
    })
    rekom_df = rekom_df.join(produk_unique[['Nama Asuransi', 'Premi Asuransi']], on='produk_idx')

    # Ambil hanya 3 kolom yang diminta
    return rekom_df.sort_values('Rating', ascending=False).head(top_n)[
        ['Nama Asuransi', 'Premi Asuransi', 'Rating']
    ].reset_index(drop=True)

In [10]:
recommend_hybrid_by_user_premi('U0012', premi_user=600000)

,Nama Asuransi,Premi Asuransi,Rating
0,BCA Asuransi Rumah Tinggal,774000.0000,4.776614
1,AXA Mandiri Asuransi Rumahku Plus,657428.5714,4.763715
2,ACA Asuransi Rumah Idaman (ASRI),657428.5714,4.715252
3,BCA Asuransi Kebakaran,769000.0000,4.707764
4,AXA Mandiri Asuransi Rumahku Plus,657428.5714,4.706834
5,BCA Asuransi Kebakaran,769000.0000,4.694042
6,BCA Asuransi Properti Komersial,774000.0000,4.661593
7,BCA Asuransi Rumah Tinggal,774000.0000,4.633005
8,BCA Asuransi Properti Komersial,774000.0000,4.590656
9,BCA Asuransi Kebakaran,769000.0000,4.560251


In [11]:
# user baru
recommend_hybrid_by_user_premi('user_baru_123', premi_user=750000)

User user_baru_123 tidak ditemukan, gunakan cold-start berbasis premi user.


,Nama Asuransi,Premi Asuransi,Rating
0,BCA Asuransi Rumah Tinggal,774000.0000,4.812531
1,AXA Mandiri Asuransi Rumahku Plus,657428.5714,4.802804
2,ACA Asuransi Rumah Idaman (ASRI),657428.5714,4.753964
3,BCA Asuransi Kebakaran,769000.0000,4.743214
4,AXA Mandiri Asuransi Rumahku Plus,657428.5714,4.737425
5,BCA Asuransi Kebakaran,769000.0000,4.729319
6,BCA Asuransi Properti Komersial,774000.0000,4.698507
7,BCA Asuransi Rumah Tinggal,774000.0000,4.673823
8,BCA Asuransi Properti Komersial,774000.0000,4.619500
9,AXA Mandiri Asuransi Rumahku Plus,657428.5714,4.600095


In [12]:
model.save('rekomendasi_properti.h5')

In [13]:
import joblib

# Simpan scaler fitur
joblib.dump(scaler_premi , "scaler_rekomendasi_properti.pkl")

['scaler_rekomendasi_properti.pkl']

# INFERENSI

In [1]:
from rekomendasi_properti import recommend_hybrid_by_user_premi

user_id = 'U9999'
premi_input = 750000

rekomendasi = recommend_hybrid_by_user_premi(user_id, premi_input, top_n=10)
rekomendasi

User U9999 tidak ditemukan, gunakan cold-start berbasis premi user.


,Nama Asuransi,Premi Asuransi,Rating
0,BCA Asuransi Rumah Tinggal,774000.0000,4.812531
1,AXA Mandiri Asuransi Rumahku Plus,657428.5714,4.802804
2,ACA Asuransi Rumah Idaman (ASRI),657428.5714,4.753964
3,BCA Asuransi Kebakaran,769000.0000,4.743214
4,AXA Mandiri Asuransi Rumahku Plus,657428.5714,4.737425
5,BCA Asuransi Kebakaran,769000.0000,4.729319
6,BCA Asuransi Properti Komersial,774000.0000,4.698507
7,BCA Asuransi Rumah Tinggal,774000.0000,4.673823
8,BCA Asuransi Properti Komersial,774000.0000,4.619500
9,AXA Mandiri Asuransi Rumahku Plus,657428.5714,4.600095
